In [1]:
# Работа с табличными данными
import pandas as pd
import numpy as np

# Преобразование признаков
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report

# Визуализация
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

# Пайплайн
from sklearn.base import BaseEstimator, TransformerMixin

from IPython.display import Markdown as md

from motorica.utils import *
from motorica.pipeline import *

In [2]:
a = np.array([100, 0, 100, 0, 0, 100, 0, 100])
print(np.gradient(a))

w = 3
for i in range(a.shape[0] - w + 1):
    slice = a[i: i + w]
    print((slice[-1] - slice[0]) / 2)

[-100.    0.    0.  -50.   50.    0.    0.  100.]
0.0
0.0
-50.0
50.0
0.0
0.0


## Конструирование признаков

### Текущая пронация

Класс-преобразователь `PronationPredictor` для предсказания метки пронации и добавления ее в датафрейм в качестве признака берем из собственного модуля `motorica.pipeline`.

In [3]:
md(PronationPredictor.__doc__.replace("    ", ""))


Класс-преобразователь для добавления в данные признака `"act_pronation_pred"` – метки пронации.
В размеченных нами данных метка пронации уже присутствует и называется `"act_pronation"`. 
Метка принимает одно из трех возможных значений: 
- `0` - ладонь вверх
- `1` - ладонь вбок
- `2` - ладонь вниз

Метка сформирована на основании файлов описания протокола и в тестовой выборке не может быть 
использована в качестве признака (поскольку в реальных данных никаких описаний протоколов 
уже не будет).

Применеие же данного класса поволяет ввести признак пронации в тестовые данных 
и использовать его для предсказания жеста (после ***one-hot* кодирования**)

### Параметры объекта класса

**features**: *List[str], default=cols_acc + cols_gyr*<br>cписок признаков для предсказания пронации
(по умолчанию берутся каналы ACC и GYR)

**pron_col**: *str, default='act_pronation'*<br>название столбца с истиным значением метки пронации

**predicted_pron_col**: *str, default='act_pronation_pred'*<br>название столбца для предсказанной метки<br>
(в обучающем датафрейме новый столбец будет полностью аналогичен исходному столбцу с разметкой)

**model**: *default=LogisticRegression()*<br>модель (необученная), используемая для предсказания метки

**scaler**: *default=MinMaxScaler()*<br>объект-преобразователь (необученный) для шкалирования признаков

### Методы
Данный класс реализует стандартые методы классов-преобразователей *scikit-learn*:

`fit()`, `fit_transform()` и `transform()` и может быть использован как элемент пайплайна.


In [4]:
def pronation_prediction_f1(montages, features, name='f1'):
    
    pron_predictor = PronationPredictor(features)

    f1_macro = []

    for montage in montages:
        X_train, X_test, _, _ = read_train_and_test(montage, features + ['act_pronation'])
        pron_predictor.fit(X_train)
        X_test = pron_predictor.transform(X_test)
        cls_res = classification_report(
            X_test['act_pronation'], X_test['act_pronation_pred'], 
            output_dict=True
        )
        f1_macro.append(np.round(cls_res['macro avg']['f1-score'], 3))

    return pd.DataFrame({
        name: f1_macro, 
        'montage': montages
    })

In [5]:
meta_info = read_meta_info("marked/selected_montages.csv")

acc_gyr_result = pronation_prediction_f1(meta_info.index, cols_acc + cols_gyr)
acc_gyr_result['features'] = 'ACC + GYR'

acc_result = pronation_prediction_f1(meta_info.index, cols_acc)
acc_result['features'] = 'ACC only'

fig_data = pd.concat([acc_gyr_result, acc_result], axis=0).sort_values('f1')


In [6]:
px.bar(
    fig_data, x='f1', y='montage', 
    color='features', barmode='group', 
    width=900, height=800,
    color_discrete_map={'ACC only': 'skyblue', 'ACC + GYR': 'steelblue'},
    title='Предсказание пронации: <b><i>f1 macro</i></b> на тестовых выборках'
)

### Использование лаговых признаков

In [7]:
from collections import deque

a = np.array([1, 2, 4, 7, 11, 16, 1])

print(np.gradient(a).tolist())
print(np.gradient(np.gradient(a)).tolist())

class Gradient2():
    def __init__(self, spacing=1):
        self.spacing = spacing
        self.w = spacing + 2
        self.val = deque(maxlen=self.w)
        self.grad1 = deque(maxlen=self.w)
    def act(self, v):
        self.val.append(v)
        if len(self.val) == self.w:
            self.grad1.append((self.val[-1] - self.val[0]) / 2)
        if len(self.grad1) == self.w:
            return (self.grad1[-1] - self.grad1[0]) / 2
        return 0

gradient2 = Gradient2()
grad2 = []
for x in a:
    grad2.append(gradient2.act(x))

np.array(grad2).tolist()


[1.0, 1.5, 2.5, 3.5, 4.5, -5.0, -15.0]
[0.5, 0.75, 1.0, 1.0, -4.25, -9.75, -10.0]


[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, -4.25]

In [8]:
montage = '2023-05-07_16-54-27.palm'
X_train, X_test, y_train, y_test = read_train_and_test(
    montage, meta_info.loc[montage, 'hi_val_sensors'],
    target_col='act_label_ext'
)
X_train_scaled = MinMaxScaler().fit_transform(X_train)
grad1 = pd.Series(
    np.sum(np.abs(np.gradient(X_train_scaled, axis=0)), axis=1),
    index=X_train.index,
    name='grad'
)
grad1 = np.nan_to_num(grad1)
grad1 = pd.Series(grad1, index=X_train.index, name='grad1')
grad2 = np.gradient(grad1)
grad2 = np.nan_to_num(grad2)
grad2 *= np.abs(grad2)
grad2 = pd.Series(grad2, index=X_train.index, name='grad2')

fig_data = pd.concat([grad2 * 5, pd.DataFrame(y_train / 10)], axis=1)

fig = px.line(fig_data, width=1200, height=700).update_traces(line=dict(width=1))
fig.show()